# 0. Imports

In [190]:
# imports
import pandas as pd
import numpy as np
import datetime as dt
import time
import os
import pickle

from colorama import Fore, Style
from sklearn.pipeline import Pipeline
from joblib import dump, load

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [230]:
# environment variables
LOCAL_REGISTRY_PATH = "../training_outputs"

# prerequisite: having a "training_outputs" folder at the root of the project
! tree ../training_outputs

../training_outputs
├── metrics
│   ├── 20221123-163916.pickle
│   └── 20221123-170451.pickle
├── models
│   ├── 20221123-163916.joblib
│   └── 20221123-170451.joblib
└── params
    ├── 20221123-163916.pickle
    └── 20221123-170451.pickle

3 directories, 6 files


# 1. Build pipeline

In [192]:
# Get rid of nan values using ffil
def ffill_nan(X):
    return X.fillna(method='ffill', axis = 0)

In [207]:
# Create new datetime features
def add_datetime_features(X):
    
    # Copy X to avoid pandas warning
    X_rep = X.copy()
    
    # Handle datetime format
    datetime = pd.to_datetime(X_rep['date'] + ' ' + X_rep['time'])
    
    # Create new features using month, weekday, hour and minute
    X_rep['month'] = datetime.dt.month
    X_rep['weekday'] = datetime.dt.weekday
    X_rep['hour'] = datetime.dt.hour
    X_rep['minute'] = datetime.dt.minute
    
    # Consider periodic effects
    X_rep['month_sin'] = np.sin(2*np.pi*X_rep['month']/12)
    X_rep['month_cos'] = np.cos(2*np.pi*X_rep['month']/12)
    
    # Get rid of Datetime
    return X_rep.drop(columns=['date', 'time'])

In [208]:
# Create a pipeline to preprocess the data

n_estimators = 10

features = ['global_active_power', 'global_reactive_power', 'voltage', 'global_intensity', 'global_consumption']
datetimes = ['date', 'time']

preparator = ColumnTransformer([ 
    ('imputer', FunctionTransformer(ffill_nan), features),
    ('datetime_features_adder', FunctionTransformer(add_datetime_features), datetimes)
])

pipeline = Pipeline([
    ('preparator', preparator),
    ('std_scaler', StandardScaler()),
    ('estimator', RandomForestRegressor(n_estimators=n_estimators))
])

pipeline

Pipeline(steps=[('preparator',
                 ColumnTransformer(transformers=[('imputer',
                                                  FunctionTransformer(func=<function ffill_nan at 0x7f2fcc7ea5e0>),
                                                  ['global_active_power',
                                                   'global_reactive_power',
                                                   'voltage',
                                                   'global_intensity',
                                                   'global_consumption']),
                                                 ('datetime_features_adder',
                                                  FunctionTransformer(func=<function add_datetime_features at 0x7f2fc8ed3ee0>),
                                                  ['date', 'time'])])),
                ('std_scaler', StandardScaler()),
                ('estimator', RandomForestRegressor(n_estimators=10))])

# 3. Prepare the data to train

In [209]:
# Take just a subset of the data for now (year==2008)
data = pd.read_csv('../data/household_power_consumption.txt', sep=';', na_values='?')
data = data[data.Date.str.endswith('2008')]
data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 527040 entries, 547596 to 1074635
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date                   527040 non-null  object 
 1   time                   527040 non-null  object 
 2   global_active_power    526905 non-null  float64
 3   global_reactive_power  526905 non-null  float64
 4   voltage                526905 non-null  float64
 5   global_intensity       526905 non-null  float64
 6   sub_metering_1         526905 non-null  float64
 7   sub_metering_2         526905 non-null  float64
 8   sub_metering_3         526905 non-null  float64
dtypes: float64(7), object(2)
memory usage: 40.2+ MB


In [210]:
labels = ['sub_metering_1', 'sub_metering_2', 'sub_metering_3']
data['global_consumption'] = data[labels].sum(axis=1)

In [211]:
# Create X, y and save datetime in a separeted column
X = data.drop(columns=labels)
y = data[labels]
X.shape, y.shape

((527040, 7), (527040, 3))

In [212]:
y.isna().sum()

sub_metering_1    135
sub_metering_2    135
sub_metering_3    135
dtype: int64

In [213]:
# Remove na values from y
y = y.fillna(method='ffill', axis=0)

In [214]:
X.head()

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
547596,1/1/2008,00:00:00,1.620,0.070,241.25,6.6,18.0
547597,1/1/2008,00:01:00,1.626,0.072,241.74,6.6,18.0
547598,1/1/2008,00:02:00,1.622,0.072,241.52,6.6,18.0
547599,1/1/2008,00:03:00,1.612,0.070,240.82,6.6,18.0
547600,1/1/2008,00:04:00,1.612,0.070,240.80,6.6,18.0


In [215]:
y.head()

,sub_metering_1,sub_metering_2,sub_metering_3
547596,0.0,0.0,18.0
547597,0.0,0.0,18.0
547598,0.0,0.0,18.0
547599,0.0,0.0,18.0
547600,0.0,0.0,18.0


In [216]:
# Train/test Split /!\ For later, if we use sequential models (ARIMA, RNN): see if we consider a TS special train/test split (to keep sequence's logic)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((368928, 7), (158112, 7), (368928, 3), (158112, 3))

In [217]:
X_train.head()

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
650931,12/3/2008,18:15:00,0.290,0.068,242.57,1.2,0.0
1047360,13/12/2008,01:24:00,0.296,0.072,243.50,1.2,2.0
892799,27/8/2008,17:23:00,0.082,0.000,242.81,0.2,1.0
852767,30/7/2008,22:11:00,2.196,0.218,239.37,9.2,22.0
1017365,22/11/2008,05:29:00,0.414,0.168,242.54,1.8,1.0


# 4. Train and save the pipeline

In [218]:
# Function to save the pipeline locally
def save_pipeline(pipeline: Pipeline = None,
                  params: dict = None,
                  metrics: dict = None) -> None:
    """
    persist trained pipeline, params and metrics
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")

    print(Fore.BLUE + "\nSave pipeline to local disk..." + Style.RESET_ALL)

    # save params
    if params is not None:
        params_path = os.path.join(LOCAL_REGISTRY_PATH, "params", timestamp + ".pickle")
        with open(params_path, "wb") as file:
            pickle.dump(params, file)

    # save metrics
    if metrics is not None:
        metrics_path = os.path.join(LOCAL_REGISTRY_PATH, "metrics", timestamp + ".pickle")
        with open(metrics_path, "wb") as file:
            pickle.dump(metrics, file)

    # save pipeline
    if pipeline is not None:
        pipeline_path = os.path.join(LOCAL_REGISTRY_PATH, "models", timestamp + ".joblib")
        dump(pipeline, pipeline_path)
         
    print("\n✅ data saved locally")

    return None

In [219]:
%%time

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Mesure its performance
r2_score = pipeline.score(X_test, y_test)
#r2_score = pipeline.score(X_test[['Voltage']].dropna(), y_test.dropna())

# Save it locally
params = dict(
    # Model parameters
    n_estimators=10,

    # Package behavior
    context="train",

    # Data used to fit
    dataset_start=data.date.iloc[0],
    dataset_end=data.date.iloc[-1]
)

metrics = dict(r2_score=r2_score)
print(metrics)

save_pipeline(pipeline, params, metrics)

{'r2_score': 0.8895890490855001}

Save model to local disk...

✅ data saved locally
CPU times: user 1min 20s, sys: 129 ms, total: 1min 20s
Wall time: 1min 20s


# 5. Load the pipeline and try to predict

In [222]:
pipeline_path = os.path.join(LOCAL_REGISTRY_PATH, "models", "20221123-170451.joblib")
pipeline_loaded = load(pipeline_path)
pipeline_loaded

Pipeline(steps=[('preparator',
                 ColumnTransformer(transformers=[('imputer',
                                                  FunctionTransformer(func=<function ffill_nan at 0x7f2fcc7ea5e0>),
                                                  ['global_active_power',
                                                   'global_reactive_power',
                                                   'voltage',
                                                   'global_intensity',
                                                   'global_consumption']),
                                                 ('datetime_features_adder',
                                                  FunctionTransformer(func=<function add_datetime_features at 0x7f2fc8ed3ee0>),
                                                  ['date', 'time'])])),
                ('std_scaler', StandardScaler()),
                ('estimator', RandomForestRegressor(n_estimators=10))])

In [227]:
X_test

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
900214,1/9/2008,20:58:00,0.298,0.196,238.55,1.4,2.0
623087,22/2/2008,10:11:00,1.412,0.058,236.16,6.0,18.0
942536,1/10/2008,06:20:00,1.334,0.050,242.83,5.4,18.0
1018176,22/11/2008,19:00:00,1.474,0.124,236.55,6.2,0.0
676717,30/3/2008,16:01:00,0.402,0.058,246.29,1.8,0.0
...,...,...,...,...,...,...,...
708656,21/4/2008,20:20:00,0.356,0.126,242.92,1.4,1.0
999877,10/11/2008,02:01:00,0.354,0.076,244.87,1.6,0.0
941699,30/9/2008,16:23:00,0.344,0.122,238.64,1.4,0.0
880746,19/8/2008,08:30:00,0.174,0.152,240.62,0.8,1.0


In [226]:
y_pred = pd.DataFrame(pipeline_loaded.predict(X_test), columns=labels)
y_pred['datetime'] = pd.to_datetime(X_test['date'] + ' ' + X_test['time'])
y_pred

,sub_metering_1,sub_metering_2,sub_metering_3,datetime
0,0.0,1.3,0.7,NaT
1,0.0,0.4,17.6,NaT
2,0.0,0.0,18.0,NaT
3,0.0,0.0,0.0,NaT
4,0.0,0.0,0.0,NaT
...,...,...,...,...
158107,0.0,0.2,0.8,NaT
158108,0.0,0.0,0.0,NaT
158109,0.0,0.0,0.0,NaT
158110,0.0,0.0,1.0,NaT
